In [1]:
import pandas as pd
import os
import shutil
from tqdm import tqdm


In [ ]:
df = pd.read_csv("D:\CTH_CTP.rdvdata\DB-CTH_CTP.csv")

In [ ]:
df.head()

In [ ]:

cta_acc_set = set((df['Accession number']))
full_acc_set = set(os.listdir("D:\CTH"))

In [ ]:
len(cta_acc_set), len(full_acc_set)

In [ ]:
full_acc_set = set(os.listdir("D:\CTH"))

In [ ]:

# Define the source and destination directories
source_dir = 'D:\CTH_archive\DICOM'
destination_dir = 'D:\\CTH_archive\\CBV\\'

# Ensure the destination directory exists, create if not
if not os.path.exists(destination_dir):
    os.makedirs(destination_dir)

# Iterate through each accession number in the set
for acc in tqdm(cta_acc_set):
    source_path = os.path.join(source_dir, str(acc))  # Path to the directory named by the accession number
    destination_path = os.path.join(destination_dir, str(acc))

    # Check if the directory exists before trying to copy
    if os.path.exists(source_path):
        # Check if the destination directory already exists to avoid 'FileExistsError'
        if not os.path.exists(destination_path):
            shutil.copytree(source_path, destination_path)
            print(f"Directory {source_path} copied successfully to {destination_path}.")
        else:
            print(f"Destination directory {destination_path} already exists.")
    else:
        print(f"Source directory {source_path} does not exist.")


In [ ]:
import os
import pydicom
import shutil
from tqdm import tqdm

def rename_folders_with_patient_name(base_path):
    log = []  # Collect logs for troubleshooting

    for accession_dir in tqdm(next(os.walk(base_path))[1]):
        full_path = os.path.join(base_path, accession_dir)
        patient_name_found = False
        log.append(f"Processing {accession_dir}")

        for root, dirs, files in os.walk(full_path):
            for file in files:
                if file.lower().endswith('.dcm'):
                    dicom_path = os.path.join(root, file)
                    try:
                        dicom = pydicom.dcmread(dicom_path, stop_before_pixels=True)
                        patient_name = str(dicom.PatientName)
                        parts = patient_name.split('^')
                        if len(parts) > 1:
                            patient_name = '_'.join(parts)
                        patient_name = ''.join(e for e in patient_name if e.isalnum() or e == '_')
                        new_folder_path = os.path.join(base_path, patient_name)

                        if full_path != new_folder_path:
                            if not os.path.exists(new_folder_path):
                                shutil.move(full_path, new_folder_path)
                                log.append(f"Renamed {full_path} to {new_folder_path}")
                                patient_name_found = True
                                break
                            else:
                                log.append(f"Target folder {new_folder_path} already exists.")
                        else:
                            log.append(f"No need to rename {full_path}.")
                    except Exception as e:
                        log.append(f"Error processing {dicom_path}: {str(e)}")
            if patient_name_found:
                break

    # Print log entries
    for entry in log:
        print(entry)

# Example usage
base_path = "D:\\CTH_archive\\CBV_RENAME"
rename_folders_with_patient_name(base_path)


In [ ]:
import os
import pydicom
from collections import defaultdict
from tqdm import tqdm
def list_dicom_series_by_patient(base_path):
    # Dictionary to store series descriptions organized by patient name
    patient_series = defaultdict(list)

    # Iterate over each directory in the base path
    for patient_dir in tqdm(os.listdir(base_path)):
        patient_path = os.path.join(base_path, patient_dir)
        if os.path.isdir(patient_path):  # Ensure it is a directory
            for root, dirs, files in os.walk(patient_path):
                for file in files:
                    if file.lower().endswith('.dcm'):
                        dicom_path = os.path.join(root, file)
                        try:
                            # Load DICOM file
                            dicom = pydicom.dcmread(dicom_path, stop_before_pixels=True)
                            series_description = dicom.get('SeriesDescription', 'No Description')
                            # Add series description to the list for the patient
                            patient_series[patient_dir].append(series_description)
                        except Exception as e:
                            print(f"Error processing {dicom_path}: {e}")

    # Output the results
    for patient, descriptions in patient_series.items():
        print(f"Patient: {patient}")
        unique_descriptions = set(descriptions)  # Remove duplicates
        for description in unique_descriptions:
            print(f"  Series Description: {description}")

    return patient_series

# Example usage
base_path = "D:\\CTH_archive\\CTA"
dicom_series_by_patient = list_dicom_series_by_patient(base_path)

# Given the dictionary `dicom_series_by_patient` and a list of terms to ignore
ignore_terms = [
    "SCAN DOC", "No Description", "Cor", "Sag", "MIP", "Dose",
    "Bayer", "Smart", "SCANNED", "offline", "Patient", "Scout", "Screen", "SUMMARY", "SCANDOC"
    "reformat", "LVO", "monitoring", "AIDOC", "LAT", "1 CE", "OLIVE", "Topo", "ISOTROPIC", "THINS"
]

# Convert the ignore terms to uppercase for case-insensitive comparison
ignore_terms = [term.upper() for term in ignore_terms]

# Iterate through the dictionary and remove descriptions containing any ignore terms
for patient in list(dicom_series_by_patient.keys()):
    # Use a set to store unique series descriptions
    unique_series = set()
    for description in dicom_series_by_patient[patient]:
        if not any(ignore_term in description.upper() for ignore_term in ignore_terms):
            unique_series.add(description)
    # Update the dictionary with unique filtered series
    dicom_series_by_patient[patient] = list(unique_series)

# Now, dicom_series_by_patient is updated with unique series descriptions
# Print the updated list to confirm
for patient, series in sorted(dicom_series_by_patient.items()):
    print(f"Patient: {patient}")
    for description in sorted(series):  # Sorting to have a consistent output order
        print(f"  Series Description: {description}")


In [1]:
from extract_utils import DICOMSeriesExtractor

desired_series = {
    "VIZ CBV"
}

ignore_terms = [
#    "SCAN DOC", "No Description", "Cor", "Sag", "MIP", "Dose", "Bayer", "Smart",
#    "SCANNED", "offline", "Patient", "Scout", "Screen", "SUMMARY", "SCANDOC", "reformat",
#    "LVO", "monitoring", "AIDOC", "LAT", "1 CE", "OLIVE", "Topo", "ISOTROPIC", "THINS"
]
extractor = DICOMSeriesExtractor(desired_series, ignore_terms)
base_path = "D:\CTH_archive\DICOM"
extractor.validate_dicom_series(base_path)


Starting validation of DICOM series...
Processing patient directory: ALFORD_BARBARA 4024996
Validated series for ALFORD_BARBARA 4024996: {'VIZ CBV'}
Processing patient directory: ALLAH_MAJUSTICE 2621774
Validated series for ALLAH_MAJUSTICE 2621774: {'VIZ CBV'}
Processing patient directory: BATTLE_MARIA 8399298
Validated series for BATTLE_MARIA 8399298: {'VIZ CBV'}
Processing patient directory: BAUM_ROBERT 6035108
Validated series for BAUM_ROBERT 6035108: {'VIZ CBV'}
Processing patient directory: BILLIPS_JAMES 6272788
Validated series for BILLIPS_JAMES 6272788: {'VIZ CBV'}
Processing patient directory: BOGER_DAVID_S 2532249
Validated series for BOGER_DAVID_S 2532249: {'VIZ CBV'}
Processing patient directory: BROWN_ANTHONY F341525
Validated series for BROWN_ANTHONY F341525: {'VIZ CBV'}
Processing patient directory: CAMPAGNA_HARRY_D E304749
Validated series for CAMPAGNA_HARRY_D E304749: {'VIZ CBV'}
Processing patient directory: CANIGLIA_ROBERT 2363078
Validated series for CANIGLIA_ROBERT 

In [ ]:
from extract_utils import DICOMSeriesExtractor
target_base = "D:\\CTH_archive\\CBV_NIFTI"

extractor.copy_dicom(target_base)

In [ ]:
import os 
len(os.listdir(target_base))

In [ ]:
import nibabel as nib
import matplotlib.pyplot as plt
from ipywidgets import interact, IntSlider

def scroll_through_nifti(file_path):
    # Load the NIfTI file
    nifti = nib.load(file_path)
    data = nifti.get_fdata()

    # Define a function to plot a slice
    def plot_slice(slice_no):
        plt.figure(figsize=(8, 8))
        plt.imshow(data[:, :, slice_no], cmap='gray')
        plt.axis('off')
        plt.title(f'Slice {slice_no}')
        plt.show()

    # Create an interactive slider for slice selection
    interact(plot_slice, slice_no=IntSlider(min=0, max=data.shape[2] - 1, step=1, value=data.shape[2] // 2))

# Example usage in a Jupyter Notebook
file_path = "D:\CTH_archive\CTA_STRIPPED\SCOUFARAS_DESPINA.nii"
scroll_through_nifti(file_path)


In [ ]:
from image_utils import ImageVisualizer, show_progress
DATASET_NUM = "041"
FOLD_NUM = 0

show_progress(fr"D:\nnUNet_results\Dataset{DATASET_NUM}_Perfusion\nnUNetTrainer__nnUNetPlans__3d_fullres\fold_{FOLD_NUM}\progress.png")

In [2]:
from image_utils import convert_series_to_nifti
convert_series_to_nifti("D:\CTH_archive\CBF", "D:\CTH_archive\CBF_NIFTI")

RuntimeError: Exception thrown in SimpleITK ImageSeriesReader_Execute: D:\a\1\sitk\Code\IO\src\sitkImageSeriesReader.cxx:131:
sitk::ERROR: File names information is empty. Cannot read series.

In [ ]:
import os

def collect_dicom_files(base_path):
    """Collect all DICOM files organized by patient name."""
    dicom_files_by_patient = {}

    for patient_dir in os.listdir(base_path):
        patient_path = os.path.join(base_path, patient_dir)
        if os.path.isdir(patient_path):
            dicom_files = [os.path.join(root, file) for root, dirs, files in os.walk(patient_path) for file in files if file.lower().endswith('.dcm')]
            dicom_files_by_patient[patient_dir] = dicom_files

    return dicom_files_by_patient

In [ ]:
base_path = 'D:\\CTH_archive\\DICOM'
dicom_files_by_patient = collect_dicom_files(base_path)

# Print the DICOM files for each patient
for patient, dicom_files in dicom_files_by_patient.items():
    print(f"PATIENT: {patient}")
    for file in dicom_files:
        print(file)

In [8]:
import os
import pydicom

def validate_dicom_series(base_path):
    """Validate DICOM series across all patients in the specified directory."""
    print("Starting validation of DICOM series...")
    for patient_dir in os.listdir(base_path):
        patient_path = os.path.join(base_path, patient_dir)
        if os.path.isdir(patient_path):
            print(f"Processing patient directory: {patient_dir}")
            all_series_descriptions = set()  # Track all series descriptions found
            dicom_files = [os.path.join(root, file) for root, dirs, files in os.walk(patient_path) for file in files if file.lower().endswith('.dcm')]
            for dicom_file in dicom_files:
                try:
                    dicom = pydicom.dcmread(dicom_file, stop_before_pixels=True)
                    series_description = getattr(dicom, 'SeriesDescription', 'Unknown').strip().upper()
                    all_series_descriptions.add(series_description)  # Collect all descriptions
                except Exception as e:
                    print(f"Error processing {dicom_file}: {e}")

            #print(f"Available series for patient '{patient_dir}': {all_series_descriptions}")  # Print all series descriptions

            if "VIZ CBF" not in map(str.upper, all_series_descriptions):
                print(f"Error: No 'VIZ CBF' series identified for patient '{patient_dir}'.")
            elif list(map(str.upper, all_series_descriptions)).count("VIZ CBF") > 1:
                print(f"Error: More than one 'VIZ CBF' series identified for patient '{patient_dir}'.")
            else:
                print(f"Validated 'VIZ CBF' series for {patient_dir}.")

    print("Validation complete.")

# Call the function
base_path = 'D:\\CTH_archive\\DICOM'
validate_dicom_series(base_path)

Starting validation of DICOM series...
Processing patient directory: ALFORD_BARBARA 4024996
Validated 'VIZ CBF' series for ALFORD_BARBARA 4024996.
Processing patient directory: ALLAH_MAJUSTICE 2621774
Validated 'VIZ CBF' series for ALLAH_MAJUSTICE 2621774.
Processing patient directory: BATTLE_MARIA 8399298
Validated 'VIZ CBF' series for BATTLE_MARIA 8399298.
Processing patient directory: BAUM_ROBERT 6035108
Validated 'VIZ CBF' series for BAUM_ROBERT 6035108.
Processing patient directory: BILLIPS_JAMES 6272788
Validated 'VIZ CBF' series for BILLIPS_JAMES 6272788.
Processing patient directory: BOGER_DAVID_S 2532249
Validated 'VIZ CBF' series for BOGER_DAVID_S 2532249.
Processing patient directory: BROWN_ANTHONY F341525
Validated 'VIZ CBF' series for BROWN_ANTHONY F341525.
Processing patient directory: CAMPAGNA_HARRY_D E304749
Validated 'VIZ CBF' series for CAMPAGNA_HARRY_D E304749.
Processing patient directory: CANIGLIA_ROBERT 2363078
Validated 'VIZ CBF' series for CANIGLIA_ROBERT 236307

In [1]:
import os
import pydicom
import shutil
from tqdm import tqdm

def copy_dicom(base_path, target_base):
    """Copy 'VIZ CBF' DICOM series to the specified directory."""
    print("Starting copying of DICOM files...")
    patient_dirs = os.listdir(base_path)
    for patient_dir in tqdm(patient_dirs, desc="Copying DICOM files"):
        patient_path = os.path.join(base_path, patient_dir)
        if os.path.isdir(patient_path):
            dicom_files = [os.path.join(root, file) for root, dirs, files in os.walk(patient_path) for file in files if file.lower().endswith('.dcm')]
            for dicom_file in dicom_files:
                try:
                    dicom = pydicom.dcmread(dicom_file, stop_before_pixels=True)
                    series_description = getattr(dicom, 'SeriesDescription', 'Unknown').strip().upper()
                    if series_description == "VIZ CBF":
                        # Create a directory for the patient if it doesn't exist
                        patient_name = patient_dir.split()[0]  # Get just the patient name
                        patient_dir = os.path.join(target_base, patient_name)
                        os.makedirs(patient_dir, exist_ok=True)

                        # Copy the DICOM file to the patient's directory
                        output_file = os.path.join(patient_dir, os.path.basename(dicom_file))
                        shutil.copyfile(dicom_file, output_file)
                except Exception as e:
                    print(f"Failed to copy DICOM file {dicom_file} for patient {patient_dir}: {e}")

    print("All DICOM copying completed.")

# Call the function
base_path = 'D:\\CTH_archive\\DICOM'
target_base = 'D:\\CTH_archive\\CBF'
copy_dicom(base_path, target_base)

Starting copying of DICOM files...


Copying DICOM files: 100%|██████████| 113/113 [04:40<00:00,  2.48s/it]

All DICOM copying completed.


In [4]:
import os
import SimpleITK as sitk

def convert_series_to_nifti(input_directory, output_file):
    reader = sitk.ImageSeriesReader()
    dicom_names = reader.GetGDCMSeriesFileNames(input_directory)
    reader.SetFileNames(dicom_names)
    image_series = reader.Execute()
    
    # Convert to numpy array to manipulate the pixel data directly
    img_array = sitk.GetArrayFromImage(image_series)
        
    # Convert the numpy array back to a SimpleITK Image
    processed_image = sitk.GetImageFromArray(img_array)
    processed_image.SetSpacing(image_series.GetSpacing())
    processed_image.SetOrigin(image_series.GetOrigin())
    processed_image.SetDirection(image_series.GetDirection())

    # Write the processed image as a NIfTI file
    sitk.WriteImage(processed_image, output_file)

def convert_all_series_to_nifti(base_path, target_base):
    """Convert all DICOM series in the base directory to NIfTI format and save them in the target directory."""
    print("Starting conversion of DICOM series to NIfTI format...")
    for patient_dir in os.listdir(base_path):
        patient_path = os.path.join(base_path, patient_dir)
        if os.path.isdir(patient_path):
            output_file = os.path.join(target_base, f"{patient_dir}.nii")
            try:
                convert_series_to_nifti(patient_path, output_file)
                print(f"Converted DICOM series from {patient_dir} to {output_file}")
            except Exception as e:
                print(f"Failed to convert DICOM series for patient {patient_dir}: {e}")

    print("All conversions completed.")

# Call the function
base_path = 'D:\\CTH_archive\\CBF'
target_base = 'D:\\CTH_archive\\CBF_NIFTI'
convert_all_series_to_nifti(base_path, target_base)

Starting conversion of DICOM series to NIfTI format...
Converted DICOM series from ALFORD_BARBARA to D:\CTH_archive\CBF_NIFTI\ALFORD_BARBARA.nii
Converted DICOM series from ALLAH_MAJUSTICE to D:\CTH_archive\CBF_NIFTI\ALLAH_MAJUSTICE.nii
Converted DICOM series from BATTLE_MARIA to D:\CTH_archive\CBF_NIFTI\BATTLE_MARIA.nii
Converted DICOM series from BAUM_ROBERT to D:\CTH_archive\CBF_NIFTI\BAUM_ROBERT.nii
Converted DICOM series from BILLIPS_JAMES to D:\CTH_archive\CBF_NIFTI\BILLIPS_JAMES.nii
Converted DICOM series from BOGER_DAVID_S to D:\CTH_archive\CBF_NIFTI\BOGER_DAVID_S.nii
Converted DICOM series from BROWN_ANTHONY to D:\CTH_archive\CBF_NIFTI\BROWN_ANTHONY.nii
Converted DICOM series from CAMPAGNA_HARRY_D to D:\CTH_archive\CBF_NIFTI\CAMPAGNA_HARRY_D.nii
Converted DICOM series from CANIGLIA_ROBERT to D:\CTH_archive\CBF_NIFTI\CANIGLIA_ROBERT.nii
Converted DICOM series from CARDIN_PAUL to D:\CTH_archive\CBF_NIFTI\CARDIN_PAUL.nii
Converted DICOM series from CHANG_WAH_KONG to D:\CTH_archiv